In [1]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [2]:
# If working in Colab, mount your drive
from google.colab import drive
drive.mount('/content/drive')

# Direct to your project folder.
%cd /content/drive/Othercomputers/Computer1/dl-robocop

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/Computer1/dl-robocop


In [3]:
# Installing requirements
!pip install -r requirements.txt

In [4]:
#from datasets import load_dataset
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
#import evaluate
#import gensim
import transformers
import nltk
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data.dataset import random_split
#import jsonlines

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# BART MODEL (STUDENT)
model_name = "facebook/bart-large"
student_tokenizer = AutoTokenizer.from_pretrained(model_name)
student_tokenizer.pad_token_id = student_tokenizer.eos_token_id
student_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

#model.load_state_dict(torch.load('bart_test.pth'))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# PARADETOX MODEL (TEACHER)
#from transformers import BartForConditionalGeneration, AutoTokenizer
#base_model_name = 'facebook/bart-base'
#model_name = 'SkolkovoInstitute/bart-base-detox'
#teacher_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
#teacher_model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

In [6]:
# POLITENESS JUDGE
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# load tokenizer and model weights
classifier_tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
toxicity_clasifier_model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from helpers import format_data

data_path = 'data/'
dataset = pd.read_csv(data_path + 'paradetox.tsv', sep='\t')

# Define the maximum length and other parameters
max_length = 38
tensor_dataset = format_data(max_length, student_tokenizer, dataset)

# Define the size of the training set
train_size = int(0.8 * len(tensor_dataset))
test_size = len(tensor_dataset) - train_size

# Split the dataset into training and test sets
train_dataset, test_dataset = random_split(tensor_dataset, [train_size, test_size])

# Defining parameters for training
batch_size = 16
lr = 3e-5
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size = 16, shuffle = False)
optimizer = torch.optim.Adam(student_model.parameters(), lr=lr)
num_epochs = 1

In [8]:
from helpers import evaluate, calculate_toxicity_score, policy_sampling, train_on_paradetox
train_on_paradetox(student_model = student_model,
                   judge_model = toxicity_clasifier_model,
                   tokenizer = student_tokenizer,
                   judge_tokenizer = classifier_tokenizer,
                   optimizer =optimizer,
                   dataloader = train_dataloader,
                   test_dataloader = test_dataloader,
                   num_epochs = num_epochs,
                   alpha = 0.5,
                   device = device,
                   lambda_tox = 1,
                   lambda_bert = 1)


 1/597


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 2/597


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 3/597


KeyboardInterrupt: 

In [ ]:
prompt = "Rewrite the following toxic input into non-toxic version. Let's break the input down step by step to rewrite the non-toxic version. You should first think about the expanation of why the input text is toxic. Then generate the detoxic output. You must preserve the original meaning as much as possible.\nInput: "

for (idx,batch) in enumerate(test_dataloader):
  #input = prompt+input+"\n"
  model.eval()
  input_ids, detoxified_ids, mask = batch

  input_ids = input_ids.to(device)
  detoxified_ids = detoxified_ids.to(device)
  mask = mask.to(device)

  outputs = model.generate(input_ids = input_ids, attention_mask = mask, max_new_tokens = 50, do_sample=False)
  print('Toxic sentence: ', ' '.join([tokenizer.decode(i, skip_special_tokens = True) for i in input_ids[0]]))
  print(f'Detoxified sentence:\n {tokenizer.decode(outputs[0], skip_special_tokens = True)}')
  print('Gold: ', ' '.join([tokenizer.decode(i, skip_special_tokens = True) for i in detoxified_ids[0]]))
  print()
  if idx == 10:
    break